# This Template is created to make grading fair and straightforward. Anything not in the place as mentioned in the template would not be graded.

<font color='red'> # NOTE: We would run the notebook through a Plagiarism Checker. If it is found to be copied, your work would not be graded, and the incident would be highlighted to NYU Authorities. </font>

# Import Library and Dataset

In [4]:
import numpy
import sklearn
# So On.......

# PART I: Preprocessing

#### Handling missing values. (If ANY)

In [ ]:
#Code...

#### Feature Datatype Conversion From Numeric to categoric and Vice-versa. (If ANY)

In [ ]:
#Code...

#### Feature Reduction or extraction. (If ANY)

In [ ]:
#Code...

#### Any other Pre-processing Used. (Give the name along with the code.)

In [ ]:
#Code...

# PART II: Classification

### Model 1:
Model Name:-----------<br>
Evaluation method and metric used Name:-----------<br>
Name of the Hyperparameter used:--------------......<br>


In [5]:
#Code...

### Model 2:
Model Name:-----------<br>
Evaluation method and metric used Name:-----------<br>
Name of the Hyperparameter used:--------------......<br>


In [ ]:
#Code...

### Model 3:
Model Name:-----------<br>
Evaluation method and metric used Name:-----------<br>
Name of the Hyperparameter used:--------------......<br>


In [ ]:
#Code...

# PART III: Best Hypothesis:
Model Name:------------<br>
Reason:--------------<br>
Hyper-parameter Value:-----------<br>
